### Too slow to run 5-s sampled data...

In [1]:
import argparse
import datetime
import os.path
import time
import sys
import numpy as np
import itertools
import pandas as pd
import backtrader as bt
    
class Position(bt.observer.Observer):
    '''This observer keeps track of the current portfolio value in the broker
    including the cash
    Params:
      - ``fund`` (default: ``None``)
        If ``None`` the actual mode of the broker (fundmode - True/False) will
        be autodetected to decide if the returns are based on the total net
        asset value or on the fund value. See ``set_fundmode`` in the broker
        documentation
        Set it to ``True`` or ``False`` for a specific behavior
    '''
    lines = ('value',)

    plotinfo = dict(plot=True, subplot=True)

    def next(self):
        self.lines[0][0] = self._owner.broker.getposition(self.data).size

class CustomPandasData(bt.feeds.PandasData):
    lines = ('best_bid', 'best_ask', 'imbalance05', 'imbalance10', 'imbalance25', 'imbalance50', 'imbalance75', 'imbalance100')
    params = (
        ('nullvalue', float('NaN')),
#         ('dtformat', '%Y-%m-%d %H:%M:%S'),

        ('datetime', None),
        ('open', 0),
        ('high', 1),
        ('low', 2),
        ('close', 3),
        ('volume', 4),
        ('openinterest', -1),
        ('best_bid', 5),
        ('best_ask', 6),
        ('imbalance05', 7),
        ('imbalance10', 8),
        ('imbalance25', 9),
        ('imbalance50', 10),
        ('imbalance75', 11),
        ('imbalance100', 12),
    )

In [2]:
class St(bt.Strategy):
    def __init__(self):
        self.buy_order = None
        self.sell_order = None
    
    def next(self):
        # { 'A': 152.2752233323304, 'B': 130.81676890098976, 'half_spread': 0.005462219490893135, 'imbalance': 9 }
        A = 152.2752233323304
        B = 130.81676890098976
        half_spread = 0.005462219490893135
        max_position = 100
        
        x = self.position.size
        skew = B * x / max_position * -1
        quote_mid_price = self.data.close[0] + A * self.data.imbalance25[0] / 100000000 + skew
        tick_size = 0.5
        
        new_bid = np.minimum(np.minimum(np.round(quote_mid_price * (1 - half_spread) / tick_size) * tick_size, self.data.close[0] - tick_size), self.data.best_bid[0])
        new_ask = np.maximum(np.maximum(np.round(quote_mid_price * (1 + half_spread) / tick_size) * tick_size, self.data.close[0] + tick_size), self.data.best_ask[0])

        if x >= max_position:
            new_bid = np.nan
        if x <= -max_position:
            new_ask = np.nan
        
        if self.buy_order is not None:
            self.buy_order.cancel()
            self.buy_order = None
        if np.isfinite(new_bid):
            self.buy_order = self.buy(price=new_bid, size=1, exectype=bt.Order.Limit)
            
        if self.sell_order is not None:
            self.sell_order.cancel()
            self.sell_order = None
        if np.isfinite(new_ask):
            self.sell_order = self.sell(price=new_ask, size=1, exectype=bt.Order.Limit)            
            
def run():
    datakwargs = dict()
#     fromdate = datetime.datetime.strptime(args.fromdate, '%Y-%m-%d')
#     datakwargs['fromdate'] = fromdate
#     todate = datetime.datetime.strptime(args.todate, '%Y-%m-%d')
#     datakwargs['todate'] = todate

    df = pd.read_pickle('../../data/data')
    df = df[(df.index > '2021-01-01') & (df.index < '2021-1-31')]
    data = CustomPandasData(dataname=df, timeframe=bt.TimeFrame.Seconds, compression=5)

    cerebro = bt.Cerebro()
    cerebro.adddata(data)

    cash = 1000000
    
    cerebro.broker.set_cash(cash)
    cerebro.broker.setcommission(commission=-0.00025, margin=False)
    cerebro.addstrategy(St)

    cerebro.addobserver(bt.observers.BuySell)
    cerebro.addobserver(bt.observers.Value, timeframe=bt.TimeFrame.Minutes, compression=60)
    cerebro.addobserver(Position, timeframe=bt.TimeFrame.Minutes, compression=60)
    
#     cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Minutes, compression=60)
#     cerebro.addanalyzer(bt.analyzers.Calmar, timeframe=bt.TimeFrame.Minutes, compression=60)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Minutes, compression=60)
    
    stats = cerebro.run(stdstats=False)
    cerebro.plot(style='bar')

    return stats

In [ ]:
%%time
stats = run()

In [ ]:
stats[0].analyzers.drawdown.get_analysis()

In [ ]:
hourly_returns = [i for i in x[0].analyzers.timereturn.get_analysis().values()]

In [ ]:
np.mean(hourly_returns) / np.std(hourly_returns) * np.sqrt(24 * 252)